#Inplement the simple rnn

Implement the simple rnn using pytorch.
Lean the sin curve with noise to RNN to plot the curve line by predecit the future.

# Make train data

Make the train data by add the randam noise to sin curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sin_x = np.linspace(-2*np.pi,2*np.pi)
sin_y = np.sin(sin_x) + 0.1*np.random.randn(len(sin(x)))
plt.plot(sin_x,sin_y)
plt.show()

# data pretreatment

reshape the input and correct data to match the RNN. To be able to predict the next value from time sieries, the time sieries used as input and the correct answer is the value one after it.


In [ ]:
import torch
from torch.utils.data import DataLoader

n_time = 10
n_sample = len(sin_x) - n_time

input_data = np.zeros((n_sample, n_time,1))
correct_data = np.zeros((n_sample,1))
for i in range(n_sample) :
  input_data[i] = sin_y[i:i+n_time].reshape(-1,1)
  correct_data[i] = sin_y[i+n_time:i+n_time+1]

input_date = torch.tensor(input_data, dtype=torch.float)
correct_data = torch.tensor(correct_data, dtype= torch.float)
dataset = torch.utils.data.TensorDataset(input_data,correct_data)

train_loader = DataLoader(dataset,batch_size=8,shuffle=True)

# Construct the model

Construct the model as an inheritance of the nn.Module.
RNN can inplement using nn.Rnn

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.Rnn(
        input_size = 1,
        hidden_size = 64,
        batch_first = True,
    )
    self.fc = nn.Linear(64,1)

  def forward(self,x):
    y_rnn,h = self,rnn(x,None)
    y = self.fc (y_rnn[:,-1,:])
    return y

  net = Net()
  print(net)

# Learning

Learning the model. Take out the data to train and evaluate.
Using the trained model,curves are generated by adding the forcast results using the most recent time series one after the other to the time series.

In [ ]:
from torch import optim

#平均二乗誤差関数
loss_fnc = nn.MSELoss()

#最適化アルゴリズム
optimizer = optim.SGD(net.parameters(),lr=0.01)

# loss log
record_loss_train = []

#Learning
for i in range(50):
  net.train()
  loss_train = 0
  for j , (x,t) in enumerate(train_loader): #take out minibath(x,t)
    y  = net(x)
    loss = loss_fnc(y,t)
    loss_train += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /= j+1 
  record_loss_train.append(loss_train)

  if i%2 == 0:

    print("Epoch: ",i,"Loss_Train", loss_train)
    predicted = list(input_data[0],reshape(-1))
    for i in range(n_sample) :
      x = torch.tensor(predicted[-n_time:])
      x = x.reshape(1,n_time,1)
      y=net(x)
      predicted.append(y[0].item())

    plt.plot(range(len(sin_y)),sin_y,label="Correct")
    plt.plot(range(len(predicted)),predicted,label="Predicted")
    plt.legend()
    plt.show()

# Error trends

Plot the error trends graph by testing and training data

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)),record_loss_train,label="Train")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()